In [1]:
    # Import packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, date, time, timedelta

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor


import init
import baseline_model as bm

RSEED=42

In [2]:
# Load data
df_train = init.load_data("train")
df_train.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target,...,STA_hour,DURATION,plane,DATOP_DEP_count,DATOP_ARR_count,DEP_lat,DEP_lon,ARR_lat,ARR_lon,STD_hour_category
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12:55:00,ATA,TU 32AIMN,260.0,...,12,145.0,32A,2,41,33.367500,-7.58997,36.851002,10.227200,morning
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16:55:00,ATA,TU 31BIMO,20.0,...,16,110.0,31B,1,39,45.630600,8.72811,36.851002,10.227200,noon
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,0.0,...,6,155.0,32A,36,2,36.851002,10.22720,41.262222,28.727778,night
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,0.0,...,17,170.0,736,11,2,33.875000,10.77550,47.153198,-1.610730,noon
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15:50:00,ATA,TU 320IMU,22.0,...,15,80.0,320,40,2,36.851002,10.22720,36.691002,3.215410,noon


In [3]:
# Drop useless features
df_train = df_train[["DEPSTN","ARRSTN","STATUS","airline_code","STD_month","STD_day","DURATION","plane","DATOP_DEP_count","DATOP_ARR_count","STD_hour_category","target"]]

In [4]:
# Create dummy variables
categorical_cols = ['STATUS','DEPSTN','ARRSTN','airline_code','plane',"STD_hour_category"]
df_analysis = pd.get_dummies(df_train,columns=categorical_cols,drop_first=True,dtype=int)

In [5]:
# Split data
y = df_analysis["target"]
X = df_analysis.drop("target", axis=1)

# Split into train and test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RSEED)

In [ ]:
# XG Boost
model = XGBRegressor()

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [17]:
from sklearn.metrics import make_scorer

score_func = make_scorer(bm.get_accuracy)

In [ ]:
param_grid = 

grid_logreg = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='score_func', 
                           verbose=5, n_jobs=-1)

In [15]:
# Make predictions on test set
y_predict = model.predict(X_test)
y_predict_train = model.predict(X_train)

In [16]:
print("Accuracy for the train data:",bm.get_accuracy(y_test,y_predict,10))
print("Accuracy for the test data:",bm.get_accuracy(y_train,y_predict_train,10))

Accuracy for the train data: 0.2213567023693606
Accuracy for the test data: 0.2267521387336842
